In [1]:
from flightplotting import plotsec, plot_regions, plotdtw
from flightdata import Flight, State
from pathlib import Path
from ardupilot_log_reader import Ardupilot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import geometry as g
from plots import plot_controller
from parsing import make_controller_field, get_last_log

np.set_printoptions(suppress=True)

fl = Flight.from_log(get_last_log(), ['TSPD', 'TRLL', 'TPIT', 'TYAW', 'PINF'], 
    TRLL = make_controller_field('trll'),
    TPIT = make_controller_field('tpit'),
    PINF = dict(
        pinf_id='id',
        pinf_cmd='cmd',
        pinf_stage='stage',
        pinf_pitch='pitch',
    )
)


st = State.from_flight(fl) \
    .label(stage=fl.pinf_stage.fillna('auto')) \
        .label(mode=fl.pinf_id.fillna(0).astype(int).map(lambda x: f'para_{x}'))


def plot_para(_fl, _st):
    return go.Figure(
        data=[
            go.Scatter(x=_st.data.index,  y=-_st.dw / 9.81, name='load factor'),
            #go.Scatter(x=_fl.data.index,  y=-_fl.data.tpit_Curr / 9.81 , name='current acc'),
            go.Scatter(x=_st.data.index,  y=_st.z , name='height', yaxis='y2'),
            go.Scatter(x=_st.data.index,  y=_fl.air_speed , name='air speed', yaxis='y3'),
            #go.Scatter(x=_fl.data.index,  y=_fl.data.tpit_Total , name='pitch_demand', yaxis='y4'),
        ],
        layout=dict(
            title='parabola info',
            yaxis=dict(title='load factor, g'),
            yaxis2=dict(title='height, m', overlaying='y', anchor='free', autoshift=True),
            yaxis3=dict(title='speed, m/s', overlaying='y', anchor='free', autoshift=True),
            yaxis4=dict(title='pitch', overlaying='y', anchor='free', autoshift=True),
        )
    )



#plot_regions(st, 'mode').show()
#plotsec(stt, nmodels=20, scale=1).show()

sts = []
fls=[]
for i, _st in  st.get_label_subset(stage='para').data.groupby('mode'):
    sts.append(State(_st))
    fls.append(fl.slice_time_flight(slice(_st.t.iloc[0],_st.t.iloc[-1])))



In [2]:

colors = px.colors.qualitative.Plotly

fig = go.Figure()
for i, _st in enumerate(sts):
    bx = _st.att.transform_point(g.PX())
    pitch = np.arcsin(bx.z / abs(bx))
    fig.add_trace(go.Scatter(x=st.t, y=_st.att.inverse().transform_point(_st.acc).z, line=dict(color=colors[i], dash='solid')))
    fig.add_trace(go.Scatter(x=st.t, y=pitch, line=dict(color=colors[i], dash='dash'), yaxis='y2'))
    fig.add_trace(go.Scatter(x=st.t, y=abs(_st.vel), line=dict(color=colors[i], dash='dot'), yaxis='y3'))
    
    
fig.update_layout(
    yaxis2=dict(title='pitch, rad', overlaying='y', anchor='free', autoshift=True),
    yaxis3=dict(title='speed, m/s', overlaying='y', anchor='free', autoshift=True),
)
fig.show()

In [3]:
from flightplotting import vectors
_st = sts[-1]
_fl = fls[-1]
fig = plotsec(_st, nmodels=5, scale=1)

fig.add_traces(vectors(30, _st, -_st.att.inverse().transform_point(_st.acc)))


In [4]:
fig = go.Figure(data=[
    go.Scatter(x=_st.t, y=_st.rvel.y, name='rvel'),
    go.Scatter(x=_st.t, y=_st.att.inverse().transform_point(g.PZ(9.81)).z / abs(_st.vel), name='des rvel'),
    go.Scatter(x=_st.t, y=np.radians(_fl.data.pinf_pitch), name='pitch'),
])
fig.show()

In [5]:
_fl.pinf_pitch, _st.t


(            pinf_pitch
 time_index            
 0.000000    -13.381105
 0.039984    -13.303529
 0.079968    -13.268166
 0.119952    -13.253227
 0.159936    -13.291140
 ...                ...
 5.399506    -10.291511
 5.439490    -10.237261
 5.479474    -10.134420
 5.519458    -10.134420
 5.559442    -10.134420
 
 [140 rows x 1 columns],
 array([162.405012, 162.444996, 162.48498 , 162.524964, 162.564948,
        162.604932, 162.644916, 162.6849  , 162.724884, 162.764868,
        162.804852, 162.844836, 162.88482 , 162.924804, 162.964788,
        163.004772, 163.044756, 163.08474 , 163.124724, 163.164708,
        163.204692, 163.244676, 163.28466 , 163.324644, 163.364628,
        163.404612, 163.444596, 163.48458 , 163.524564, 163.564548,
        163.604532, 163.644516, 163.6845  , 163.724484, 163.764468,
        163.804452, 163.844436, 163.88442 , 163.924404, 163.964388,
        164.004372, 164.044356, 164.08434 , 164.124324, 164.165141,
        164.205125, 164.245109, 164.285093, 164.3

In [6]:
px.line(_st.att.inverse().transform_point(g.PZ(9.81))).show()
px.line(_st.att.transform_point(g.PZ(9.81))).show()